In [3]:
import numpy as np
a = np.array([1,2,3,4])

In [15]:
import os
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon, Circle
import numpy as np
import dota_utils as util
from collections import defaultdict # 使用 defaultdict
import cv2

In [13]:
#The code is used for visulization, inspired from cocoapi
#  Licensed under the Simplified BSD License [see bsd.txt]

import os
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon, Circle
import numpy as np
import dota_utils as util
from collections import defaultdict # 使用 defaultdict
import cv2

def _isArrayLike(obj):
    if type(obj) == str:
        return False
    return hasattr(obj, '__iter__') and hasattr(obj, '__len__')

class DOTA:
    def __init__(self, basepath):
        self.basepath = basepath
        self.labelpath = os.path.join(basepath, 'labelTxt')
        self.imagepath = os.path.join(basepath, 'images')
        self.imgpaths = util.GetFileFromThisRootDir(self.labelpath)
        self.imglist = [util.custombasename(x) for x in self.imgpaths]
        self.catToImgs = defaultdict(list)# defaultdict(type): 
                                          # self.catToImgs为dict，对没有val的的key自动配一个空list
                                          # such like:
                                          # {'ship': ['P0706', 'P0706',...],...}
            
        self.ImgToAnns = defaultdict(list)
        self.createIndex() # 初始化操作
    """
    createIndex方法：
    生成catToImgs字典，key为类别，val为imgid（list）
    生成catToAnns字典，key为imgid，val为包含的目标
    """
    def createIndex(self):
        for filename in self.imgpaths:
            objects = util.parse_dota_poly(filename)
            imgid = util.custombasename(filename)
            self.ImgToAnns[imgid] = objects
            for obj in objects:
                cat = obj['name']
                self.catToImgs[cat].append(imgid)
                
    """ 
    getImgIds方法：
    输入需要查找的目标类别，返回所有包含指定类别（多目标要求包括所有种类）的图像的ID
    """
    def getImgIds(self, catNms=[]):
        """
        :param catNms: category names
        :return: all the image ids contain the categories
        """
        catNms = catNms if _isArrayLike(catNms) else [catNms]
        if len(catNms) == 0:  # catNums为空则返回所有img的id 
            return self.imglist
        else:
            imgids = []
            for i, cat in enumerate(catNms): # enumerate: 同时列出一个可迭代对象的数据和下标
                if i == 0: # 查找第一类时，将imgids初始化为所有包含此类目标的图片名set（图片名不重复）
                    imgids = set(self.catToImgs[cat])
                else:      # 在其中查找包含后续类别的图片
                           # 意味着图片需要包含catNms指定的所有种类，才会被选中
                    imgids &= set(self.catToImgs[cat])
        return list(imgids)

    def loadAnns(self, catNms=[], imgId = None, difficult=None):
        """
        :param catNms: category names
        :param imgId: the img to load anns
        :return: objects
        """
        catNms = catNms if _isArrayLike(catNms) else [catNms]
        objects = self.ImgToAnns[imgId]
        if len(catNms) == 0:
            return objects
        outobjects = [obj for obj in objects if (obj['name'] in catNms)]
        return outobjects
    """
    showAnns方法：
    传入目标类型，图片id，
    """
    def showAnns(self, objects, imgId, range):
        """
        :param catNms: category names
        :param objects: objects to show
        :param imgId: img to show
        :param range: display range in the img
        :return:
        """
        img = self.loadImgs(imgId)[0]
        plt.imshow(img)
        plt.axis('off')

        ax = plt.gca()
        ax.set_autoscale_on(False)
        polygons = []
        color = []
        circles = []
        r = 5
        for obj in objects:
            c = (np.random.random((1, 3)) * 0.6 + 0.4).tolist()[0]
            poly = obj['poly']
            polygons.append(Polygon(poly))
            color.append(c)
            point = poly[0]
            circle = Circle((point[0], point[1]), r)
            circles.append(circle)
        p = PatchCollection(polygons, facecolors=color, linewidths=0, alpha=0.4)
        ax.add_collection(p)
        p = PatchCollection(polygons, facecolors='none', edgecolors=color, linewidths=2)
        ax.add_collection(p)
        p = PatchCollection(circles, facecolors='red')
        ax.add_collection(p)
    """
    loadImgs方法：
    传入imgids的list，返回图片序列(并不显示）
    """
    def loadImgs(self, imgids=[]):
        """
        :param imgids: integer ids specifying img
        :return: loaded img objects
        """
        print('isarralike:', _isArrayLike(imgids))
        imgids = imgids if _isArrayLike(imgids) else [imgids]
        print('imgids:', imgids)
        imgs = []
        for imgid in imgids:
            filename = os.path.join(self.imagepath, imgid + '.png')
            print('filename:', filename)
            img = cv2.imread(filename)
            imgs.append(img)
        return imgs

# if __name__ == '__main__':
#     examplesplit = DOTA('examplesplit')
#     imgids = examplesplit.getImgIds(catNms=['plane'])
#     img = examplesplit.loadImgs(imgids)
#     for imgid in imgids:
#         anns = examplesplit.loadAnns(imgId=imgid)
#         examplesplit.showAnns(anns, imgid, 2)

In [14]:
import dota_utils as util

In [16]:
x = DOTA('example')

In [18]:
imgids = x.getImgIds(catNms=['ship', 'storage-tank'])
imgid = imgids[1]
img = x.loadImgs(imgid)[0]

isarralike: False
imgids: ['P2709']
filename: example/images/P2709.png
